<a href="https://colab.research.google.com/github/OmarShafie/MNIST-HBKU/blob/master/bert_tensorflow_2_huggingface_transformers_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Dec  9 17:51:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    12W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! pip install -q kaggle
from google.colab import files
# Upload the API Token
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge
!ls -l

import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm.notebook import tqdm

!pip install transformers

subm_dir = 'sample_submission.csv'

label_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

!unzip train.csv.zip
!unzip test.csv.zip
!unzip test_labels.csv.zip

Train_Data = pd.read_csv('train.csv')
Test_Data = pd.read_csv('test.csv')
Test_Labels = pd.read_csv('test_labels.csv')
Test_Labels = Test_Labels.set_index('id')

Train_Data.head()

Saving kaggle.json to kaggle.json
 84% 22.0M/26.3M [00:00<00:00, 110MB/s]
100% 26.3M/26.3M [00:00<00:00, 105MB/s]
  0% 0.00/1.46M [00:00<?, ?B/s]
100% 1.46M/1.46M [00:00<00:00, 97.9MB/s]
  0% 0.00/1.39M [00:00<?, ?B/s]
100% 1.39M/1.39M [00:00<00:00, 201MB/s]
 21% 5.00M/23.4M [00:00<00:00, 20.0MB/s]
100% 23.4M/23.4M [00:00<00:00, 67.4MB/s]
total 53908
-rw-r--r-- 1 root root       66 Dec  9 17:51 kaggle.json
drwxr-xr-x 1 root root     4096 Dec  2 22:04 sample_data
-rw-r--r-- 1 root root  1459715 Dec  9 17:51 sample_submission.csv.zip
-rw-r--r-- 1 root root 24577258 Dec  9 17:51 test.csv.zip
-rw-r--r-- 1 root root  1527605 Dec  9 17:51 test_labels.csv.zip
-rw-r--r-- 1 root root 27619914 Dec  9 17:51 train.csv.zip
     |████████████████████████████████| 1.4MB 7.5MB/s 
     |████████████████████████████████| 890kB 26.1MB/s 
     |████████████████████████████████| 2.9MB 44.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=ab0dc6b7fffd7913c

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

#Using the bert model for tokenization

tokenize = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
Max_ln = 160

def Sentences_Tokenizer(sentences, tokenize, max_seq_len = Max_ln):
    tokenized_sentences = []

    for sentence in tqdm(sentences):
        tokenized_sentence = tokenize.encode(
                            sentence,                  
                            add_special_tokens = True,
                            max_length = max_seq_len,
                    )
        tokenized_sentences.append(tokenized_sentence)
    return tokenized_sentences

def Attention_mask(tokenized_padded_sentence):
    attention_mask = []

    for sentence in tokenized_padded_sentence:
        mask = [int(token_id > 0) for token_id in sentence]
        attention_mask.append(mask)

    return np.asarray(attention_mask)

In [4]:
#Doing the task of Tokenization as well as Truncation and Padding

ids_input = Sentences_Tokenizer(Train_Data['comment_text'], tokenize, Max_ln)

ids_input = pad_sequences(ids_input, maxlen=Max_ln, dtype="long", value=0, truncating="post", padding="post")

attention_mask = Attention_mask(ids_input)

labels =  Train_Data[label_columns].values
train_inputs , validation_inputs = ids_input , ids_input
train_labels, validation_labels = labels , labels
train_masks, validation_masks = attention_mask, attention_mask
_, _ = labels, labels

Train_size = len(train_inputs)
Validation_size = len(validation_inputs)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [5]:
batch = 32
Epochs = 2

def create_dataset(data_tuple, epochs=1, batch_size=32, buffer_size=10000, train=True):
    dataset = tf.data.Dataset.from_tensor_slices(data_tuple)
    if train:
        dataset = dataset.shuffle(buffer_size=buffer_size)
    dataset = dataset.repeat(epochs)
    dataset = dataset.batch(batch)
    if train:
        dataset = dataset.prefetch(1)
    
    return dataset

train_dataset = create_dataset((train_inputs, train_masks, train_labels), epochs=Epochs, batch_size=batch)
validation_dataset = create_dataset((validation_inputs, validation_masks, validation_labels), epochs=Epochs, batch_size=batch)

In [6]:
#BERT model is loaded from library of transformers

from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Flatten
from transformers import TFBertModel

class BertClassifier(tf.keras.Model):    
    def __init__(self, bert: TFBertModel, num_classes: int):
        super().__init__()
        self.bert = bert
        self.classifier = Dense(num_classes,kernel_regularizer=regularizers.l2(1e-2), activation='sigmoid')
        
    @tf.function
    def call(self, ids_input, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None):
        output = self.bert(ids_input,
                               attention_mask=attention_mask,
                               token_type_ids=token_type_ids,
                               position_ids=position_ids,
                               head_mask=head_mask)
        cls_output = output[1]
        cls_output = self.classifier(cls_output)
                
        return cls_output

model = BertClassifier(TFBertModel.from_pretrained('bert-base-uncased'), len(label_columns))

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
from transformers import create_optimizer
import time

steps_epoch = Train_size // batch
steps_validation = Validation_size // batch

#Definition of the Loss Function
obj_loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
train_loss = tf.keras.metrics.Mean(name='train_loss')
val_loss = tf.keras.metrics.Mean(name='test_loss')

#Definition of the Optimizer
pre_steps = steps_epoch // 3
tot_steps = steps_epoch * Epochs - pre_steps
optimizer,_ = create_optimizer(init_lr=2e-5, num_train_steps=tot_steps, num_warmup_steps=pre_steps)

train_metrics = [tf.keras.metrics.AUC() for i in range(len(label_columns))]
validation_metrics = [tf.keras.metrics.AUC() for i in range(len(label_columns))]

@tf.function
def train_step(model, token_ids, masks, labels):
    labels = tf.dtypes.cast(labels, tf.float32)

    with tf.GradientTape() as tape:
        predictions = model(token_ids, attention_mask=masks)
        loss = obj_loss(labels, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    grads_var = [(gradients[i], model.trainable_variables[i]) for i in range(len(gradients))]
    optimizer.apply_gradients(grads_var)

    train_loss(loss)

    for i, auc in enumerate(train_metrics):
        auc.update_state(labels[:,i], predictions[:,i])
        
@tf.function
def validation_step(model, token_ids, masks, labels):
    labels = tf.dtypes.cast(labels, tf.float32)

    predictions = model(token_ids, attention_mask=masks, training=False)
    v_loss = obj_loss(labels, predictions)

    val_loss(v_loss)
    for i, auc in enumerate(validation_metrics):
        auc.update_state(labels[:,i], predictions[:,i])
                                              
def train(model, train_dataset, val_dataset, train_steps_per_epoch, val_steps_per_epoch, epochs):
    for epoch in range(epochs):
        print('=' * 50, f"EPOCH {epoch}", '=' * 50)

        start = time.time()

        for i, (token_ids, masks, labels) in enumerate(tqdm(train_dataset, total=train_steps_per_epoch)):
            train_step(model, token_ids, masks, labels)
            if i % 1000 == 0:
                print(f'\nTrain Step: {i}, Loss: {train_loss.result()}')
                for i, label_name in enumerate(label_columns):
                    print(f"{label_name} roc_auc {train_metrics[i].result()}")
                    train_metrics[i].reset_states()
        
        for i, (token_ids, masks, labels) in enumerate(tqdm(val_dataset, total=val_steps_per_epoch)):
            validation_step(model, token_ids, masks, labels)

        print(f'\nEpoch {epoch+1}, Validation Loss: {val_loss.result()}, Time: {time.time()-start}\n')

        for i, label_name in enumerate(label_columns):
            print(f"{label_name} roc_auc {validation_metrics[i].result()}")
            validation_metrics[i].reset_states()

        print('\n')

train(model, train_dataset, validation_dataset, train_steps_per_epoch=steps_epoch, val_steps_per_epoch=steps_validation, epochs=Epochs)

================================================== EPOCH 0 ==================================================



Train Step: 0, Loss: 0.7577337026596069
toxic roc_auc 0.28125
severe_toxic roc_auc 0.0
obscene roc_auc 0.36666667461395264
threat roc_auc 0.0
insult roc_auc 0.4333333373069763
identity_hate roc_auc 0.8548387289047241

Train Step: 1000, Loss: 0.16619296371936798
toxic roc_auc 0.88563072681427
severe_toxic roc_auc 0.8418320417404175
obscene roc_auc 0.886060357093811
threat roc_auc 0.7126516699790955
insult roc_auc 0.8926512598991394
identity_hate roc_auc 0.7814172506332397


In [ ]:
ids_input_Test = Sentences_Tokenizer(Test_Data['comment_text'], tokenize, Max_ln)
ids_input_Test = pad_sequences(ids_input_Test, maxlen=Max_ln, dtype="long", value=0, truncating="post", padding="post")
test_attention_mask = Attention_mask(ids_input_Test)

In [ ]:
batch_test = 32
test_steps = len(Test_Data) // batch_test

test_dataset = create_dataset((ids_input_Test, test_attention_mask), batch_size=batch_test, train=False, epochs=1)

df_submission = pd.read_csv(subm_path, index_col='id')

for i, (token_ids, masks) in enumerate(tqdm(test_dataset, total=test_steps)):
    sample_ids = Test_Data.iloc[i*batch_test:(i+1)*batch_test]['id']
    predictions = model(token_ids, attention_mask=masks).numpy()

    df_submission.loc[sample_ids, label_columns] = predictions

df_submission.to_csv('submission.csv')
!kaggle competitions submit -c jigsaw-toxic-comment-classification-challenge -f submission.csv -m "sb_comment"